In [86]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

In [76]:
X_train = pd.read_csv('data/X_train.csv', index_col='id')
X_test = pd.read_csv('data/X_test.csv', index_col='id')
y_train = pd.read_csv('data/y_train.csv', index_col='id')
y_test = pd.read_csv('data/y_test.csv', index_col='id')

In [77]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40095 entries, 45522 to 9914
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   amount_tsh             40095 non-null  float64
 1   funder                 40095 non-null  object 
 2   gps_height             40095 non-null  int64  
 3   installer              40095 non-null  object 
 4   longitude              40095 non-null  float64
 5   latitude               40095 non-null  float64
 6   num_private            40095 non-null  int64  
 7   basin                  40095 non-null  object 
 8   region                 40095 non-null  object 
 9   region_code            40095 non-null  int64  
 10  district_code          40095 non-null  int64  
 11  population             40095 non-null  int64  
 12  public_meeting         40095 non-null  object 
 13  scheme_management      40095 non-null  object 
 14  permit                 40095 non-null  object 
 15 

In [78]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13365 entries, 70336 to 10514
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   amount_tsh             13365 non-null  float64
 1   funder                 13365 non-null  object 
 2   gps_height             13365 non-null  int64  
 3   installer              13365 non-null  object 
 4   longitude              13365 non-null  float64
 5   latitude               13365 non-null  float64
 6   num_private            13365 non-null  int64  
 7   basin                  13365 non-null  object 
 8   region                 13365 non-null  object 
 9   region_code            13365 non-null  int64  
 10  district_code          13365 non-null  int64  
 11  population             13365 non-null  int64  
 12  public_meeting         13365 non-null  object 
 13  scheme_management      13365 non-null  object 
 14  permit                 13365 non-null  object 
 15

In [79]:
y_train.value_counts(normalize=True)

status_group           
functional                 0.543010
non functional             0.384138
functional needs repair    0.072852
dtype: float64

In [80]:
y_test.value_counts(normalize=True)

status_group           
functional                 0.542985
non functional             0.384138
functional needs repair    0.072877
dtype: float64

In [81]:
y_train = y_train.status_group

In [82]:
y_test = y_test.status_group

In [84]:
X_train_cat = X_train.select_dtypes('object')
X_train_num = X_train.select_dtypes(['float64', 'int64'])

cat_pipe = Pipeline([('ohe', OneHotEncoder(drop='first', sparse=False))])
num_pipe = Pipeline([('ss', StandardScaler())])

transformer = ColumnTransformer([('categorical', cat_pipe, X_train_cat.columns), 
                                  ('numerical', num_pipe, X_train_num.columns)])

In [74]:
#transformer_pipe = Pipeline([('transformer', transformer)])
#transformed_data = transformer_pipe.fit_transform(X_train)

{'categorical': Pipeline(steps=[('ohe', OneHotEncoder(drop='first', sparse=False))]),
 'numerical': Pipeline(steps=[('ss', StandardScaler())])}

In [36]:
logreg_pipe = Pipeline([('trans', transformer), ('logreg', LogisticRegression(max_iter=1000))])
logreg_pipe.fit(X_train, y_train)

Pipeline(steps=[('trans',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 sparse=False))]),
                                                  Index(['funder', 'installer', 'basin', 'region', 'public_meeting',
       'scheme_management', 'permit', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_typ...
       'quality_group', 'quantity', 'source', 'source_type', 'source_class',
       'waterpoint_type', 'waterpoint_type_group'],
      dtype='object')),
                                                 ('numerical',
                                                  Pipeline(steps=[('ss',
                                                     

In [85]:
print('Accuracy Score Train:', logreg_pipe.score(X_train,y_train))
#print('Accuracy Score Test:', logreg_pipe.score(X_test,y_test))

Accuracy Score Train: 0.778775408405038


In [92]:
log_loss_logreg = cross_val_score(logreg_pipe, X_train, y_train, scoring='neg_log_loss', cv=5)
#log_loss_logreg = -log_loss_logreg.mean()
#print('Log Loss:', log_loss_logreg)

ValueError: Found unknown categories ['Kurrp', 'Cper', 'Kenyans Company', 'Kagunguli Secondary', 'Tingatinga Sec School', 'Gg', 'Tgrs', 'Rotary Club Of Chico And Moshi', 'Runda', 'Makori', 'Jwtz', 'Ngumi', 'Halimashau', 'Ermua', 'Makapuchini', 'Embasy Of Japan In Tanzania', 'Ramadhani M. Mvugalo', 'Iran Gover', 'Said Omari', 'St Magreth Church', 'Cc Motor Day 2010', 'Bgss', 'Uniseg', 'Esawa', "Ju-sarang Church' And Bugango", 'Hasawa', 'Singsinga', 'Mikumi G', 'Mahemba', 'Mgaya Masese', "Tlc/seleman Mang'ombe", 'Jafary Mbaga', 'Calvary Connect', 'Lgcgd', 'Mwalimu Omari', 'Water Aid/dwe', 'Bhws', 'Dwarf', 'Makanga', 'Kaaya', 'Mosqure', 'College', 'Shinyanga Shallow Wells', 'Makoye Masanzu', 'Shule Ya Sekondari Ipuli', 'Mp Mloka', 'Mwinjuma Mzee', 'Hasnein Murij', 'Chani', 'Shule Ya Msingi', 'Kwa Makala', 'Mbwana Omari', 'Apm', 'Rc Msufi', 'Seleman Rashid', 'Oikos E.Africa/ European Union', 'Dwe/ubalozi Wa Marekani', 'Icap', 'Salim Ahmed Salim', 'Hewawa', 'Tasaf Ii', 'Rwssp Shinyanga', 'R

In [87]:
knn_pipe = Pipeline([('trans', transformer), ('knn', KNeighborsClassifier())])
knn_pipe.fit(X_train, y_train)

Pipeline(steps=[('trans',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 sparse=False))]),
                                                  Index(['funder', 'installer', 'basin', 'region', 'public_meeting',
       'scheme_management', 'permit', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_typ...
       'quality_group', 'quantity', 'source', 'source_type', 'source_class',
       'waterpoint_type', 'waterpoint_type_group'],
      dtype='object')),
                                                 ('numerical',
                                                  Pipeline(steps=[('ss',
                                                     

In [88]:
knn_pipe.score(X_train, y_train)

0.8314752462900611

In [89]:
dummies = pd.get_dummies(X_train)
dummies.head()

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,funder_0,...,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,waterpoint_type_group_cattle trough,waterpoint_type_group_communal standpipe,waterpoint_type_group_dam,waterpoint_type_group_hand pump,waterpoint_type_group_improved spring,waterpoint_type_group_other
id,,,,,,,,,,,,,,,,,,,,,
45522,500.0,259,36.681690,-8.264460,0,5,3,500,1997,0,...,0,1,0,0,0,0,0,1,0,0
57418,0.0,0,33.326652,-3.753205,0,17,3,0,0,0,...,0,0,0,1,0,0,0,0,0,1
44776,0.0,1878,36.706876,-3.143518,0,2,2,150,1997,0,...,0,0,0,0,0,1,0,0,0,0
5960,0.0,1206,34.495673,-1.531975,0,20,2,300,2010,0,...,0,1,0,0,0,0,0,1,0,0
11266,0.0,336,39.485704,-10.602813,0,9,4,600,1982,0,...,0,0,0,0,0,1,0,0,0,0
